## Cargar y parsear archivo

In [2]:
import pandas as pd

In [1]:
url = 'https://raw.githubusercontent.com/AnaLuisaIA/antifungal-peptides/refs/heads/main/databases/APD3/anitifungal_APD3.csv'

In [3]:
df = pd.read_csv(url)

In [4]:
display(df.head())

,Sequence
0,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV
1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...
2,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC
3,GLFDIIKKIAESI
4,GLFDIIKKIAESF


In [5]:
df.shape

(1210, 1)

## Verificar duplicados

In [6]:
df["Sequence"].unique().shape

(1199,)

In [7]:
df = df.drop_duplicates().copy()

In [8]:
df.shape

(1199, 1)

## Verificar canónicos

In [9]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [10]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["Sequence"]]

In [11]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [17]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 1199


In [18]:
df_validas = df_validas.rename(columns={"Sequence": "sequence"})

In [19]:
# Agregar una columna de label (si tiene actividad antifúngica o no)
df_validas['label'] = 1

In [20]:
display(df_validas)

,sequence,label
0,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV,1
1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,1
2,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC,1
3,GLFDIIKKIAESI,1
4,GLFDIIKKIAESF,1
...,...,...
1205,FLSLIPKIAGGIAALAKHL,1
1206,FALGAVTKVLPKLFCLITRKC,1
1207,GLKDWWNKHKDKIVKVVKEMGKAGINAA,1
1208,FLPALAGIAGLLGKIF,1


In [21]:
df_validas.to_csv('APD3_labeled.csv', index=False, header=True)

## Obtener metadatos

In [22]:
import json

In [23]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [24]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [25]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [28]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "APD3_data [multipep]")

In [29]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [30]:
export_json("metadata.json", dict_metadata)